In [1]:
import os, gc
from tqdm import tqdm

from PIL import Image

def save_first_n_images_as_tiff(input_tiff, output_tiff, n=20):
    images = []
    
    with Image.open(input_tiff) as tiff_img:
        for i in range(n):
            try:
                tiff_img.seek(i)
                extracted_image = tiff_img.copy()
                images.append(extracted_image)
                print(f"Extracted image {i+1}")
            except EOFError:
                # Reached the end of the TIFF file
                break

    if images:
        images[0].save(output_tiff, save_all=True, append_images=images[1:], compression="tiff_deflate")
        print(f"Saved first {n} images as: {output_tiff}")
    else:
        print("No images found in the input TIFF file.")


# input_tiff_file = 'path/to/your/input_tiff_file.tif'
# output_image_file = 'path/to/your/output_image_file.tif'
# sum_first_n_images(input_tiff_file, output_image_file)

        
def count_frames_in_tiff(tiffpath):
    with Image.open(tiffpath) as tiff_img:
        nframes = tiff_img.n_frames
    return nframes

flatten = lambda xxlis: [x for xx in xxlis for x in xx]

In [2]:
datadir = "/media/kyohei/forAI/cryoEM-data/"

In [3]:
cryoEM_datanumber_list = [os.path.join(datadir, f) 
                    for f in os.listdir(datadir) if f.isnumeric() ]
print(cryoEM_datanumber_list)
tiffpath_list = [
    [os.path.join(c, p) for p in os.listdir(c) if ".tif" in p] 
    for c in cryoEM_datanumber_list
]
# print(tiffpath_list)

['/media/kyohei/forAI/cryoEM-data/11349', '/media/kyohei/forAI/cryoEM-data/11217', '/media/kyohei/forAI/cryoEM-data/11244', '/media/kyohei/forAI/cryoEM-data/10997', '/media/kyohei/forAI/cryoEM-data/11084', '/media/kyohei/forAI/cryoEM-data/10828', '/media/kyohei/forAI/cryoEM-data/11387', '/media/kyohei/forAI/cryoEM-data/11029', '/media/kyohei/forAI/cryoEM-data/11351', '/media/kyohei/forAI/cryoEM-data/10877']


In [4]:
tiffpath_list = flatten(tiffpath_list)

In [5]:
count_list = []
for p in tqdm(tiffpath_list):
    count_list.append(count_frames_in_tiff(p))

  0%|                                                                                                                                                                               | 0/95 [00:00<?, ?it/s]/home/kyohei/miniconda3/envs/cryoem/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94279680 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 136.53it/s]


# Frames in data: minimum 40 frames, maximum 64 frames

In [6]:
import pandas as pd
df = pd.DataFrame([count_list,tiffpath_list]).T
df.columns = ["nframes", "tiffpath"]

print(df.nframes.value_counts())

df["data_number"] = df["tiffpath"].str.split("/").apply(lambda x: x[5])

df["out_path"] = df["tiffpath"].apply(lambda t: t.replace("/cryoEM-data/","/sumTIFF/"))

df["filename"] = df["tiffpath"].str.split("/").apply(lambda x: x[-1])

nframes
40    40
64    14
50    10
60    10
59    10
63     8
48     3
Name: count, dtype: int64


In [7]:
df.tiffpath.values[0]

'/media/kyohei/forAI/cryoEM-data/11349/FoilHole_11540439_Data_11537508_11537510_20210112_134815_fractions.tiff'

# Shorten the tiff

In [13]:
# for t in df.tiffpath:
#     out = t.replace("/cryoEM-data/","/shortTIFF/")
#     outdir = "/".join(out.split("/")[:-1])
#     if not os.path.exists(outdir):
#         os.system(f"mkdir -p {outdir}")
#     if os.path.exists(out):
#         continue
        
#     assert t!=out
    
#     save_first_n_images_as_tiff(t, out)
    
#     gc.collect()


# Just summing up the tiff

In [12]:
df["sum_path"] = df["out_path"].str.replace("/shortTIFF/", "/sumTIFF/")

In [ ]:
df.to_csv("data_in_HD_2023-0511.csv", index=None)

In [20]:

import numpy as np

def sum_first_n_images(input_tiff, output_image, n=80):
    from PIL import Image
    summed_image = None

    with Image.open(input_tiff) as tiff_img:
        for i in range(n):
            try:
                tiff_img.seek(i)
                current_frame = np.array(tiff_img)
                if summed_image is None:
                    summed_image = current_frame
                else:
                    summed_image += current_frame
            except EOFError:
                # Reached the end of the TIFF file
                break

    if summed_image is not None:
        summed_image_pil = Image.fromarray(summed_image)
        summed_image_pil.save(output_image)
        print(f"Saved summed image as: {output_image}")
    else:
        print("No images found in the input TIFF file.")


In [ ]:
!mkdir 

In [27]:
for ii, row in df.iterrows():
    t = row.tiffpath
    out = t.replace("/cryoEM-data/","/sumTIFF/")
    outdir = "/".join(out.split("/")[:-1])
    
    if not os.path.exists(outdir):
        os.system(f"mkdir -p {outdir}")
    if os.path.exists(out):
        continue
        
    assert t!=out
    sum_first_n_images(t, out, n=row.nframes)
    
#     break
#     gc.collect()


/home/kyohei/miniconda3/envs/cryoem/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94279680 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540463_Data_11537508_11537510_20210112_134408_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540425_Data_11537508_11537510_20210112_131211_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540503_Data_11537502_11537504_20210112_142344_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540485_Data_11537502_11537504_20210112_150608_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540451_Data_11537505_11537507_20210112_132841_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540473_Data_11537499_11537501_20210112_143426_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540514_Data_11537511_11537513_20210112_145303_fractions.tiff
Saved summed image as: /media/kyohei/forAI/sumTIFF/11349/FoilHole_11540431_Data_11537502_11537504

Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_7-A-007_0001_Apr09_18.14.55.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_10-A-011_0002_Apr09_19.19.06.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_6-A-033_0001_Apr09_17.53.09.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_6-A-009_0006_Apr09_16.53.10.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_6-A-021_0008_Apr09_17.23.48.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_12-A-008_0002_Apr09_20.10.19.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_7-A-010_0006_Apr09_18.23.33.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_11-A-005_0005_Apr09_19.33.07.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/11351/ikko2_6-A-004_0001_Apr09_16.39.06.tif
Saved summed image as: /media/kyohei/forAI/sumTIFF/10877/FoilHole_18713375_Data_18711503_18711505_20201219_063535_fractions.tiff
Saved summed 

In [17]:
out

'/media/kyohei/forAI/sumTIFF/11349/FoilHole_11540439_Data_11537508_11537510_20210112_134815_fractions.tiff'

In [23]:
!ls {outdir}

/bin/bash: /home/kyohei/miniconda3/envs/cryoem/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ls: cannot access '/media/kyohei/forAI/sumTIFF/11349': No such file or directory
